In [31]:
import os
import sys
from pathlib import Path

nb_dir = Path(os.getcwd())

project_root = nb_dir.parents[1]

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print("project_root:", project_root)
print("sys.path[0]:", sys.path[0])

project_root: /data/ephemeral/pro-nlp-generationfornlp-nlp-13
sys.path[0]: /data/ephemeral/pro-nlp-generationfornlp-nlp-13


In [ ]:
def extract_digit_logits(
    outputs,
    generated_ids: torch.Tensor,
    digit_token_ids: list[int],
):
    """
    returns:
      digit_pos: int | None
      subset_logits: List[float] | None   # [l1, l2, ..., lk]
      chosen_digit: str | None
    """
    digit_pos = None
    chosen_token_id = None

    # 숫자 토큰이 처음 등장한 위치
    for i, tid in enumerate(generated_ids.tolist()):
        if tid in digit_token_ids:
            digit_pos = i
            chosen_token_id = tid
            break

    if digit_pos is None:
        return None, None, None

    step_logits = outputs.scores[digit_pos][0]  # (vocab,)
    subset_logits = step_logits[digit_token_ids]  # (k,)

    chosen_digit = str(digit_token_ids.index(chosen_token_id) + 1)

    return (
        digit_pos,
        subset_logits.detach().cpu().tolist(),
        chosen_digit,
    )

In [ ]:
def process_row(
    row_dict: Dict,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str = "cuda",
    max_new_tokens: int = 100,
) -> Dict:

    output = builder.build_message(row_dict)
    messages = output["messages"]

    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)

    k = int(row_dict["choices_len"])
    digit_token_ids = [
        tokenizer.encode(str(i), add_special_tokens=False)[0]
        for i in range(1, k + 1)
    ]

    input_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,
        )

    generated_ids = outputs.sequences[0][input_len:]
    full_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)

    digit_pos, digit_logits, chosen_digit = extract_digit_logits(
        outputs,
        generated_ids,
        digit_token_ids,
    )

    return {
        "id": row_dict.get("id"),
        "predicted_answer": extract_answer(full_text),  # 기존 방식
        "chosen_digit": chosen_digit,                   # logit 기반
        "digit_logits": digit_logits,                   # ⭐ 핵심
        "digit_pos": digit_pos,
        "full_output": full_text,
    }

## 시험용

In [2]:
import yaml
import pandas as pd
import torch
import re
from transformers import AutoTokenizer
from pathlib import Path
from typing import Dict, Any

from src.data.preprocessor import parse_problems_column, add_choices_len
from src.prompt.prompt_builder import PromptBuilder, PromptConfig
from src.training.model_loader import ModelConfig, load_model_inference


In [3]:
config_path = "../../config.yaml"

with open(config_path, "r") as f:
    cfg_dict = yaml.safe_load(f)

print("Config loaded successfully!")

# Model Config
model_cfg_dict = cfg_dict["model"].copy()
model_cfg_dict["use_gradient_checkpointing"] = False
model_cfg = ModelConfig(**model_cfg_dict)

# Prompt Config
prompt_dict = cfg_dict["inference"]["prompt"]
prompt_cfg = PromptConfig(
    policy=prompt_dict["policy"],
    mode="test",
    verbose=False
)

# Inference Config
inference_cfg = cfg_dict.get("inference", {})
adapter_path = inference_cfg["adapter_path"]
test_data_path = inference_cfg["test_data_path"]
max_new_tokens = inference_cfg.get("max_new_tokens", 100)

print(f"Adapter Path: {adapter_path}")
print(f"Test Data Path: {test_data_path}")
print(f"Max New Tokens: {max_new_tokens}")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

### 데이터 경로 설정
print(f"Loading test data from {project_root / test_data_path}...")
test_df = pd.read_csv(project_root / test_data_path)
test_df = parse_problems_column(test_df)
test_df = add_choices_len(test_df)

print(f"Loaded {len(test_df)} rows")
test_df.head()

print(f"Loading tokenizer from {model_cfg.model_name_or_path}...")
tokenizer = AutoTokenizer.from_pretrained(
    model_cfg.model_name_or_path,
    trust_remote_code=model_cfg.trust_remote_code,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer loaded successfully!")

print(f"Loading model from {adapter_path}...")
model = load_model_inference(model_cfg, project_root / adapter_path)
model.eval()
print("Model loaded successfully!")

builder = PromptBuilder(prompt_cfg)
print("PromptBuilder ready!")

Config loaded successfully!
Adapter Path: ./models/qwen3_14B_eng_aug2/final_model
Test Data Path: ./data/test.csv
Max New Tokens: 30
Device: cuda
Loading test data from /data/ephemeral/pro-nlp-generationfornlp-nlp-13/data/test.csv...
Loaded 869 rows
Loading tokenizer from Qwen/Qwen3-14B...
Tokenizer loaded successfully!
Loading model from ./models/qwen3_14B_eng_aug2/final_model...
Loading Base Model for Inference: Qwen/Qwen3-14B


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Loading LoRA Adapter from: /data/ephemeral/pro-nlp-generationfornlp-nlp-13/models/qwen3_14B_eng_aug2/final_model
Model loaded successfully!
PromptBuilder ready!


In [7]:
# 첫 번째 샘플 가져오기
row_dict = test_df.iloc[0].to_dict()

# Prompt 생성
output = builder.build_message(row_dict)
messages = output["messages"]

print("Messages:")
for msg in messages:
    print(f"\n[{msg['role']}]")
    print(msg['content'] if len(msg['content']) > 500 else msg['content'])

prompt_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)

print("Prompt Text:")
print(prompt_text)
print(f"\n... (total length: {len(prompt_text)} chars)")

Messages:

[system]
당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.
이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.
당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.

[user]
### 지문
사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다 .   독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의  즐거움’이 있다. 독자는 독서를 통해 책과 소통하는 즐거움을 경험한다 .  독서는   필자와 간접적으로 대화하는 소통 행위이다 .  독자는 자신이 속한   사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는  사회나 시대를 경험한다.  직접 경험하지 못했던 다양한 삶을  필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를   바라볼 수 있다.  이때 같은 책을 읽은 독자라도 독자의 배경 지식이나 관점 등의 독자 요인,  읽기 환경이나 과제 등의 상황  요인이 다르므로,  필자가 보여 주는 세계를 그대로 수용하지  않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다 . 이러한 소통은 독자가 책의 내용에 대해 질문하고 답을  찾아내는 과정에서 가능해진다.  독자는 책에서 답을 찾는  질문 ,  독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다 .   전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고,   책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로  구성할 수도 있다.  또한 후자의 경우 책에는 없는 독자의  경험에서 답을 찾을 수 있다.  이런 질문들을 풍부히 생성 하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다 . 한편 독자는 ㉠다른 독자와 소통하는 즐거움 을 경험할 수도  있다 .  책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임 이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당 한다.  비슷한 해석에 서로 공감하며 기존 인식을 강화하거나  관점의 차이를 확인하고 기존 인식을 조정하는 과정에

In [ ]:
### 실제 사용.
def extract_answer(text: str) -> str:
    numbers = re.findall(r'[1-5]', text)
    return numbers[-1] if numbers else "no"


def process_row(
    row_dict: Dict,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str = "cuda",
    max_new_tokens: int = 30,
) -> Dict:

    output = builder.build_message(row_dict)
    messages = output["messages"]
    
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    
    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)
    
    input_len = inputs["input_ids"].shape[1]
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,
        )
    
    generated_ids = outputs.sequences[0][input_len:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    step_logits = outputs.scores[-2][0]
    
    k = int(row_dict.get("choices_len", 5))
    
    topk_values, topk_indices = torch.topk(step_logits, k=k)
    probs_full = torch.softmax(step_logits, dim=-1)
    
    topk_candidates = []
    for rank, (logit_val, token_id) in enumerate(zip(topk_values, topk_indices)):
        topk_candidates.append({
            "rank": rank + 1,
            "token_id": token_id.item(),
            "token": tokenizer.decode([token_id.item()]),
            "logit": logit_val.item(),
            "prob": probs_full[token_id].item(),
        })
    
    answer = extract_answer(generated_text)
    
    return {
        "id": row_dict.get("id"),
        "answer": answer,
        "top_candidates": topk_candidates,
        "generated_text": generated_text,
        "prompt": prompt_text
    }


In [33]:
row_dict = test_df.iloc[800].to_dict()
result = process_row_with_logits(
    row_dict=row_dict,
    builder=builder,
    tokenizer=tokenizer,
    model=model,
    device=device,
    max_new_tokens=max_new_tokens,
    top_k=5
)

# 결과 출력
print(f"ID: {result['id']}")
print(f"Answer: {result['answer']}")
print(f"\nTop 5 Candidates:")
for cand in result['top5_candidates']:
    print(f"  Rank {cand['rank']}: Token='{cand['token']}' | Logit={cand['logit']:.4f} | Prob={cand['prob']:.4f}")
print(f"\nGenerated Text: {result['generated_text']}")

ID: generation-for-nlp-1565
Answer: 1

Top 5 Candidates:
  Rank 1: Token='1' | Logit=32.5000 | Prob=0.9999
  Rank 2: Token='2' | Logit=22.5469 | Prob=0.0000
  Rank 3: Token='3' | Logit=20.4219 | Prob=0.0000
  Rank 4: Token='0' | Logit=19.2188 | Prob=0.0000
  Rank 5: Token='4' | Logit=18.8125 | Prob=0.0000

Generated Text: 1


In [34]:
result

{'id': 'generation-for-nlp-1565',
 'answer': '1',
 'top5_candidates': [{'rank': 1,
   'token_id': 16,
   'token': '1',
   'logit': 32.5,
   'prob': 0.9999415874481201},
  {'rank': 2,
   'token_id': 17,
   'token': '2',
   'logit': 22.546875,
   'prob': 4.757594069815241e-05},
  {'rank': 3,
   'token_id': 18,
   'token': '3',
   'logit': 20.421875,
   'prob': 5.6821354519343e-06},
  {'rank': 4,
   'token_id': 15,
   'token': '0',
   'logit': 19.21875,
   'prob': 1.706086436570331e-06},
  {'rank': 5,
   'token_id': 19,
   'token': '4',
   'logit': 18.8125,
   'prob': 1.1364986676198896e-06}],
 'generated_text': '1',
 'prompt': '<|im_start|>system\n당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.\n이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.\n당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>\n<|im_start|>user\n### 지문\n아파트 2300여가구를 지을 수 있는 경기 수원 광교신도시 주상복합용지 C2블록이 지난 3일 최고가 매각 입찰에서 내정가격 5644억원의 133%에 달하는 7507억원을 써낸 중흥건설에 돌아갔다. 현대건설 등 대형 건설사 컨소시엄 및 호반건설과 반도건설, 아이에스동서 등 중견 건설사는 물론 개발업체인 엠디엠 등 나머지 업체들도 600

In [45]:
def process_row(
    row_dict: Dict,
    builder: PromptBuilder,
    tokenizer: AutoTokenizer,
    model: torch.nn.Module,
    device: str = "cuda",
    max_new_tokens: int = 30,
) -> Dict:

    output = builder.build_message(row_dict)
    messages = output["messages"]

    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )

    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        truncation=True,
        max_length=4096
    ).to(device)

    input_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True,
            output_scores=True,
        )

    generated_ids = outputs.sequences[0][input_len:]
    generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

    step_logits = outputs.scores[-2][0]

    k = int(row_dict.get("choices_len", 5))

    topk_values, topk_indices = torch.topk(step_logits, k=k)
    probs_full = torch.softmax(step_logits, dim=-1)

    topk_candidates = []
    for rank, (logit_val, token_id) in enumerate(zip(topk_values, topk_indices)):
        topk_candidates.append({
            "rank": rank + 1,
            "token_id": token_id.item(),
            "token": tokenizer.decode([token_id.item()]),
            "logit": logit_val.item(),
            "prob": probs_full[token_id].item(),
        })

    digit_tokens = [str(i) for i in range(1, k + 1)]
    digit_token_ids = []

    for digit in digit_tokens:
        encoded = tokenizer.encode(digit, add_special_tokens=False)
        digit_token_ids.append(encoded[0])

    digit_logits = torch.tensor([step_logits[tid].item() for tid in digit_token_ids])
    digit_probs = torch.softmax(digit_logits, dim=-1)

    top2_digit_values, top2_digit_indices = torch.topk(digit_logits, k=min(2, k))
    if k >= 2:
        logit_gap = (top2_digit_values[0] - top2_digit_values[1]).item()
    else:
        logit_gap = 0.0

    answer = extract_answer(generated_text)

    return {
        "id": row_dict.get("id"),
        "answer": answer,
        "digit_probs": digit_probs.tolist(),
        "logit_gap": logit_gap,
        "generated_text": generated_text,
        "prompt": prompt_text
    }

In [46]:
row_dict = test_df.iloc[123].to_dict()
result = process_row(
    row_dict=row_dict,
    builder=builder,
    tokenizer=tokenizer,
    model=model,
    device=device,
    max_new_tokens=max_new_tokens,
)


In [47]:
row_dict

{'Unnamed: 0': 123,
 'id': 'generation-for-nlp-123',
 'paragraph': '(가 ) 동녁 두던 밧긔 크나큰 너븐 들\uf550 만경(萬頃)  황운(黃雲)이 \uf53a 빗치 되야 잇다 중양이 거의로다 내노리 \uf537 쟈스라 블근 게  여믈고 눌은 \ue390기 \ue982져시니 술이 니글션졍 버디야 업\ue982 소냐 전가(田家)  흥미\ue285 날로 기퍼 가노매라 살여흘 긴 몰래예 밤블이 \ue64f 가시니 ㉠게 잡\ue285 아\uf550\ue38f이 그\ue566\u200b \u200b을 흣텨 잇고 호두포* 엔 구븨 예 아젹 믈이  미러오니 ㉡돗\ue38d\u200b\u200b\ue668 애내성(欸 乃聲)*이 고기 \uf48e\ue285 댱\ue999로다 경 (景 )도 됴커니와 생리(生理)라  괴로오랴 (중략) 어와 이 청경( 淸景)  갑시 이실 거시런\ue38f 적막히 다든 문애 내  분으로 드려오랴 사조(私照)* 업다 호미 거즌말 아니로다 ㉢모재(茅齋)*예 빗쵠 빗치 옥루( 玉樓)라 다\ue470 소냐 청준(淸樽)을 밧\ueb82\u200b  열고 큰 잔의 \ue1a7\ue38b 브어 ㉣죽엽(竹葉)  \ue1a7\u200b \ue285\u200b\u200b  술\ue470 \ue38f빗 조차 거후로니 표연\uf53a 일흥( 逸興)이 져기면 \ue288리로다 이적선( 李謫仙)  이려\uf537야 \ue38f을 보고 밋치닷다 춘하추동애 경물이 아름답고 주야조모( 晝夜朝暮) 애 완상이 새로오니 ㉤몸이 한가\uf537 나 귀 눈은 겨\ue470 업다 여생이 언마치리 백발이 날로 기니 세상 공명은 계륵이나 다\ue470 소냐 ⓐ강호 어조( 魚鳥)애 새 \ue587셰 깁퍼시니 옥당금마( 玉堂金馬)*의 몽혼( 夢魂)*이 섯긔엿다 초당연월( 草堂煙月) 의 시\ue477 업시 누워 이셔 촌주강어( 村酒江魚) 로 장일취( 長日醉) \ue470 원 (願 )\uf537노라 이 몸이 이러구롬도 역군은( 亦君恩) 이샷다 -신

In [ ]:
result

{'id': 'generation-for-nlp-123',
 'answer': '3',
 'digit_probs': [0.11904565244913101,
  0.0047624255530536175,
  0.7884997129440308,
  0.0571187287569046,
  0.03057345189154148],
 'logit_gap': 1.890625,
 'generated_text': '3',
 'prompt': '<|im_start|>system\n당신은 논리적인 **텍스트 분석 및 독해 전문가**입니다.\n이 문제는 오직 **제공된 지문 내의 정보**만으로 풀어야 합니다.\n당신의 외부 배경지식을 배제하고, 철저하게 지문에 명시된 내용에 근거하여 판단하십시오.<|im_end|>\n<|im_start|>user\n### 지문\n(가 ) 동녁 두던 밧긔 크나큰 너븐 들\uf550 만경(萬頃)  황운(黃雲)이 \uf53a 빗치 되야 잇다 중양이 거의로다 내노리 \uf537 쟈스라 블근 게  여믈고 눌은 \ue390기 \ue982져시니 술이 니글션졍 버디야 업\ue982 소냐 전가(田家)  흥미\ue285 날로 기퍼 가노매라 살여흘 긴 몰래예 밤블이 \ue64f 가시니 ㉠게 잡\ue285 아\uf550\ue38f이 그\ue566\u200b \u200b을 흣텨 잇고 호두포* 엔 구븨 예 아젹 믈이  미러오니 ㉡돗\ue38d\u200b\u200b\ue668 애내성(欸 乃聲)*이 고기 \uf48e\ue285 댱\ue999로다 경 (景 )도 됴커니와 생리(生理)라  괴로오랴 (중략) 어와 이 청경( 淸景)  갑시 이실 거시런\ue38f 적막히 다든 문애 내  분으로 드려오랴 사조(私照)* 업다 호미 거즌말 아니로다 ㉢모재(茅齋)*예 빗쵠 빗치 옥루( 玉樓)라 다\ue470 소냐 청준(淸樽)을 밧\ueb82\u200b  열고 큰 잔의 \ue1a7\ue38b 브어 ㉣죽엽(竹葉)  \ue1a7\u200b \ue285\u200b\u200b  술\ue470 \ue38f

: 

### 실제 검증

In [4]:
import pandas as pd
pred_df = pd.read_csv(project_root / "submission_aug2_2step.csv")
sub = pd.read_csv(project_root / "submission_aug2.csv")


In [7]:
pred_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           869 non-null    object 
 1   answer       869 non-null    object 
 2   digit_probs  869 non-null    object 
 3   logit_gap    869 non-null    float64
dtypes: float64(1), object(3)
memory usage: 27.3+ KB


In [18]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869 entries, 0 to 868
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      869 non-null    object
 1   answer  869 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 13.7+ KB


In [21]:
(pred_df.index == sub.index).all()
m = pred_df[['id','answer']].merge(
    sub[['id','answer']],
    on='id',
    how='inner',
    suffixes=('_pred','_sub')
)

a = m['answer_pred'].astype(str).str.strip()
b = m['answer_sub'].astype(str).str.strip()

(a == b).all()

False

In [22]:
diff = (a != b)
m.loc[diff, ['id','answer_pred','answer_sub']].head()

,id,answer_pred,answer_sub
517,generation-for-nlp-741,no,4


In [26]:
pred_df['logit_gap'].describe()

count    869.000000
mean       4.625827
std        3.572176
min        0.015625
25%        1.500000
50%        3.718750
75%        7.265625
max       13.312500
Name: logit_gap, dtype: float64

### 비교

In [28]:
import pandas as pd

df1 = pd.read_csv(project_root / "submission_aug2.csv")  # 1-step
df2 = pd.read_csv(project_root / "submission_aug2_2step.csv")  # 새로 생성된 2-step

merged = df1.merge(df2, on="id", suffixes=("_1step", "_2step"))
changed = merged[merged["answer_1step"] != merged["answer_2step"]]

print(f"변경된 답변: {len(changed)} / {len(merged)} ({len(changed)/len(merged)*100:.2f}%)")
if len(changed) > 0:
    print(changed[["id", "answer_1step", "answer_2step"]].head(20))

변경된 답변: 25 / 869 (2.88%)
                          id  answer_1step  answer_2step
36     generation-for-nlp-36             1             3
260   generation-for-nlp-260             1             2
278   generation-for-nlp-278             2             3
308   generation-for-nlp-308             1             2
321   generation-for-nlp-321             3             5
336   generation-for-nlp-336             1             4
367   generation-for-nlp-367             5             1
398   generation-for-nlp-398             2             5
423   generation-for-nlp-423             2             3
440   generation-for-nlp-681             2             4
456   generation-for-nlp-614             1             2
475  generation-for-nlp-2465             2             5
513   generation-for-nlp-633             2             3
601   generation-for-nlp-970             4             3
606  generation-for-nlp-1061             1             2
639  generation-for-nlp-2370             1             4
642   

In [29]:
# 전체 데이터 로드 (분석용)
df2_full = pd.read_csv(project_root / "submission_aug2_2step.csv")

# 제출용 (id, answer만)
df2_submission = df2_full[["id", "answer"]].copy()

# 제출용 파일 저장
df2_submission.to_csv(project_root / "submission_aug2_2step_submit.csv", index=False)
print(f"제출용 파일 저장: submission_aug2_2step_submit.csv")

# 분석용 파일 저장 (전체)
df2_full.to_csv(project_root / "submission_aug2_2step_full.csv", index=False)
print(f"분석용 파일 저장: submission_aug2_2step_full.csv (digit_probs, logit_gap 포함)")


제출용 파일 저장: submission_aug2_2step_submit.csv
분석용 파일 저장: submission_aug2_2step_full.csv (digit_probs, logit_gap 포함)


In [30]:
import pandas as pd

df1 = pd.read_csv(project_root / "submission_aug2.csv")  # 1-step
df2 = pd.read_csv(project_root / "submission_aug2_2step2_full.csv")  # 새로 생성된 2-step

merged = df1.merge(df2, on="id", suffixes=("_1step", "_2step"))
changed = merged[merged["answer_1step"] != merged["answer_2step"]]

print(f"변경된 답변: {len(changed)} / {len(merged)} ({len(changed)/len(merged)*100:.2f}%)")
if len(changed) > 0:
    print(changed[["id", "answer_1step", "answer_2step"]].head(20))

변경된 답변: 65 / 869 (7.48%)
                         id  answer_1step  answer_2step
31    generation-for-nlp-31             1             2
36    generation-for-nlp-36             1             3
41    generation-for-nlp-41             1             3
42    generation-for-nlp-42             5             3
64    generation-for-nlp-64             1             3
67    generation-for-nlp-67             4             5
146  generation-for-nlp-146             4             3
162  generation-for-nlp-162             2             3
194  generation-for-nlp-194             1             2
198  generation-for-nlp-198             1             2
216  generation-for-nlp-216             4             5
260  generation-for-nlp-260             1             2
262  generation-for-nlp-262             3             4
270  generation-for-nlp-270             3             2
273  generation-for-nlp-273             2             5
278  generation-for-nlp-278             2             1
281  generation-for-nlp

In [32]:
import pandas as pd

df1 = pd.read_csv(project_root / "submission_aug2.csv")  # 1-step
df2 = pd.read_csv(project_root / "submission_aug2_2step3.csv")  # 새로 생성된 2-step

merged = df1.merge(df2, on="id", suffixes=("_1step", "_2step"))
changed = merged[merged["answer_1step"] != merged["answer_2step"]]

print(f"변경된 답변: {len(changed)} / {len(merged)} ({len(changed)/len(merged)*100:.2f}%)")
if len(changed) > 0:
    print(changed[["id", "answer_1step", "answer_2step"]].head(20))

변경된 답변: 30 / 869 (3.45%)
                          id  answer_1step  answer_2step
36     generation-for-nlp-36             1             3
260   generation-for-nlp-260             1             2
278   generation-for-nlp-278             2             3
294   generation-for-nlp-294             2             5
308   generation-for-nlp-308             1             2
321   generation-for-nlp-321             3             5
336   generation-for-nlp-336             1             4
367   generation-for-nlp-367             5             1
398   generation-for-nlp-398             2             5
423   generation-for-nlp-423             2             3
440   generation-for-nlp-681             2             4
456   generation-for-nlp-614             1             2
475  generation-for-nlp-2465             2             5
513   generation-for-nlp-633             2             3
594   generation-for-nlp-926             4             2
601   generation-for-nlp-970             4             3
606  g